In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# For ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

df = pd.read_csv('/content/Copy of bankruptcy-prevention.csv')
df.columns = df.columns.str.strip() # Strip whitespace from column names
df['class'] = df['class'].map({'non-bankruptcy': 0, 'bankruptcy': 1})


feature_names = ['industrial_risk', 'management_risk', 'financial_flexibility', 'credibility', 'competitiveness', 'operating_risk'] # Now we can use names without leading spaces
X = df[feature_names]
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Classic Algorithms
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "KNN": KNeighborsClassifier()
}

results = {}

for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = (clf, acc)

# ANN Model creation function
def create_ann():
    model = Sequential()
    model.add(Dense(8, input_dim=len(feature_names), activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

ann = KerasClassifier(model=create_ann, epochs=100, batch_size=16, verbose=0)
ann.fit(X_train, y_train)
ann_acc = accuracy_score(y_test, np.round(ann.predict(X_test)))
results["ANN"] = (ann, ann_acc)

best_model_name = max(results, key=lambda x: results[x][1])
best_model, best_acc = results[best_model_name]

print(f"Best Model: {best_model_name}, Test Accuracy: {best_acc:.2f}")

# Save best model and features (classic models only, for Streamlit compatibility)
if best_model_name != "ANN":
    joblib.dump((best_model, feature_names), 'bankruptcy_model.pkl')
else:
    # Save ANN model in .h5 format and note feature_names serialization for later use
    ann.model_.save('bankruptcy_ann_model.h5')
    joblib.dump(feature_names, 'ann_feature_names.pkl')

Best Model: Logistic Regression, Test Accuracy: 1.00


In [3]:
%pip install scikeras